In [1]:
import os
import fiona
import hashlib
import boto3
import json
import numpy as np
import datacube
%matplotlib inline
import matplotlib
from datacube.api import GridWorkflow
from madmex.wrappers import gwf_query
from madmex.models import SegmentationInformation
from madmex.util import parser_extra_args
from madmex.settings import TEMP_DIR
from importlib import import_module
import gc
import shutil

In [2]:
name_of_product = 's2_10m_scl_Jalisco_some_scenes_recipe_2018'
gwf_kwargs = {'region': 'Jalisco', 'begin': '2018-01-01', 'end':'2018-12-31'}
gwf_kwargs.update(product=name_of_product)

In [3]:
iterable = gwf_query(**gwf_kwargs)

In [4]:
lista_iter = list(iterable)

In [5]:
var = lista_iter[1]

In [6]:
algorithm = 'bis'
datasource = 'sentinel2'
name = 's2_10m_some_scenes_fiona_test'
year = '2018'
extra_args = parser_extra_args(['t=30','s=0.5','c=0.7'])

In [7]:
meta, _ = SegmentationInformation.objects.get_or_create(
    algorithm=algorithm, datasource=datasource,
    parameters=json.dumps(extra_args),
    datasource_year=year,
    name=name,
)

In [8]:
dataset_name = datasource + '_%d_%d_' % (var[0][0], var[0][1]) + year

In [9]:
dataset_name

'sentinel2_28_-28_2018'

In [10]:
path = os.path.join(TEMP_DIR, 'segmentation_results')

In [11]:
if not os.path.exists(path):
    os.makedirs(path)

In [12]:
band_list = ['ndvi_mean']

In [13]:
try:
    module = import_module('madmex.segmentation.%s' % algorithm)
    Segmentation = module.Segmentation
except ImportError as e:
    raise ValueError('Invalid model argument')

In [14]:
try:
    # Load tile
    geoarray = GridWorkflow.load(var[1], measurements=band_list)
    seg = Segmentation.from_geoarray(geoarray, **extra_args)
    seg.segment()
    # Try deallocating input array
    seg.array = None
    geoarray = None
    seg.polygonize()
    # Write to shapefile
    hash = hashlib.md5(dataset_name.encode('utf-8')).hexdigest()[0:6]
    name_file = hash + '_' + dataset_name
    out_file = path + '/' + name_file
    schema = {'geometry': 'Polygon',
          'properties': [('id', 'int')]}
    with fiona.open(path, 'w', layer = name_file,
                    schema=schema,
                    driver='ESRI Shapefile',
                    crs=seg.crs) as dst:
        for feature in seg.fc:
            dst.write(feature)

    #Upload to s3 bucket
    s3 = boto3.client('s3')
    bucket_name = 'my-segmentation-bucket'
    filename = name_file + '.shp'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    filename = name_file + '.shx'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    filename = name_file + '.cpg'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    filename = name_file + '.dbf'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    filename = name_file + '.prj'
    filepath = path + '/' + filename
    s3.upload_file(filepath, bucket_name, filename)
    os.remove(filepath)
    gc.collect()
except Exception as e:
        print(e)

  Anticipated allocation: 879609KB
    ...allocated
  thresh 1
  thresh 2
  thresh 3
  thresh 4
  thresh 5
  thresh 6
  thresh 7
  thresh 8
  thresh 9
  thresh 10
  thresh 11
  thresh 12
  thresh 13
  thresh 14
  thresh 15
  thresh 16
  thresh 17
  thresh 18
  thresh 19
  thresh 20
  thresh 21
  thresh 22
  thresh 23
  thresh 24
  thresh 25
  thresh 26
  thresh 27
  thresh 28
  thresh 29
  thresh 30
